In [12]:
# FOR DEV
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pixiedust

In [13]:
pixiedust.optOut()

Pixiedust will not collect anonymous install statistics.


In [351]:
pd.options.display.max_rows = 999

## Dependencies

In [14]:
import pandas as pd
import json
import psycopg2

## Prepare for SQL connection

In [15]:
def connectToSql():
    """Connects to SQL DB via settings in config.json"""
    # Open config file
    with open('./config/config.json') as f:
       config = json.load(f)

    # Get connection params as a string
    postgres_config_string = "host=%s dbname=%s user=%s password=%s" % (config.get(
        'host'), config.get('database'), config.get('user'), config.get('passw'))

    # Connect to db 
    # If a connect cannot be made an exception will be raised here
    con = psycopg2.connect(postgres_config_string)
    
    return con

## Import dataset

In [16]:
# Connect to DB and save query as a dataframe
con = connectToSql()
query_1 = open('./sql/1_query.sql', 'r')
df = pd.read_sql(query_1.read(), con=con)

# rename SQL columns
df.rename(index=str, columns={'home_team':'homeTeam', 'guest_team':'awayTeam', 'home_score':'homeScore', 'guest_score':'awayScore', 'odds_1':'odds1', 'odds_x':'oddsX', 'odds_2':'odds2','explore_id':'exploreId'}, inplace=True)

# drop exploeID since it's not used to train the model
df.drop(columns=['exploreId'], inplace = True)

# Season cleaning
Dataset includes 'Seasons' that actually are cups, and when teams play unequal number of the features get messed up. Therefore does this function check that all teams play equal number games and drops the seasons that do not match the criteria

In [17]:
def drop_NotStdSeasons(df):
    seasonNames = df.season.unique()
    nSeasons = df.season.unique().shape[0]

    for i in range(nSeasons):
        melted_data = pd.melt(df[df.season == seasonNames[i]], value_vars=['homeTeam', 'awayTeam'], value_name='team')
        nGames = (melted_data.groupby(by=['team'])).count()
        useSeason = nGames['variable'].nunique()
    
        if useSeason == 1 and df[df.season == seasonNames[i]].shape[0] != 1:
            df.loc[df['season'] == seasonNames[i], 'useSeason'] = True
        
        else:
            df.loc[df['season'] == seasonNames[i], 'useSeason'] = False
            
        if 1 in nGames.values:
            df.loc[df['season'] == seasonNames[i], 'useSeason'] = False 
        
        
    df.drop(df.index[df['useSeason'] == False], inplace = True)
    df.drop(columns=['useSeason'], inplace = True)
    
    # sort and redefine index
    df.sort_values(by = ['season', 'date'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    seasonNames = df.season.unique()
    nSeasons = df.season.unique().shape[0]
    df.set_index(['season'], append=True,drop=True, inplace=True)
    return (seasonNames, nSeasons)

seasonNames, nSeasons = drop_NotStdSeasons(df)
 

## Helpers

In [18]:
# Sets variables for ROUNDS, GAMES and TEAMS
def get_rounds(df):
    GAMES = df.shape[0]
    TEAMS = df.homeTeam.unique().shape[0]
    ROUNDS = int(GAMES * 2 / TEAMS)
    return ROUNDS

def get_games(df):
    GAMES = df.shape[0]
    return GAMES

def get_teams(df):
    TEAMS = df.homeTeam.unique().shape[0]
    return TEAMS

## Get aggregated goals for team/round

### Helper functions

In [19]:
def get_goalsScored(frame, ROUNDS):
    # create a dictionary with team names as keys
    teams = {}
    for i in frame.groupby('homeTeam').mean().T.columns:
        teams[i] = []
        
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(frame)):
        HTGS = frame.iloc[i]['homeScore']
        ATGS = frame.iloc[i]['awayScore']
        teams[frame.iloc[i].homeTeam].append(HTGS)
        teams[frame.iloc[i].awayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    goalsScored = pd.DataFrame(data=teams, index=[i for i in range(1, ROUNDS+1)]).T
    goalsScored[0] = 0

    # Aggregate to get uptil that point
    for i in range(2, ROUNDS+1):
        goalsScored[i] = goalsScored[i] + goalsScored[i - 1]
    
    return goalsScored

# Gets the goals conceded agg arranged by teams and matchweek
def get_goalsConceded(frame, ROUNDS):
    # Create a dictionary with team names as keys
    teams = {}
    for i in frame.groupby('homeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match location.
    for i in range(len(frame)):
        ATGC = frame.iloc[i]['homeScore']
        HTGC = frame.iloc[i]['awayScore']
        teams[frame.iloc[i].homeTeam].append(HTGC)
        teams[frame.iloc[i].awayTeam].append(ATGC)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    goalsConceded = pd.DataFrame(data=teams, index=[i for i in range(1, ROUNDS+1)]).T
    goalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2, ROUNDS+1):
        goalsConceded[i] = goalsConceded[i] + goalsConceded[i - 1]
    return goalsConceded

### Main function

In [20]:
def get_aggGoals(df):
    skippedSeasons = []
    for season in range(len(seasonNames)):
        try:
            nMatchWeeks = get_rounds(df.loc[(slice(None),seasonNames[season]),:])
            nTeams = get_teams(df.loc[(slice(None),seasonNames[season]),:])

            teams = df.loc[(slice(None),seasonNames[season]),:].homeTeam.unique().tolist()
            gamesIndex = []
            for team in teams:
                df2 = df.loc[(slice(None),seasonNames[season]),:]
                indexes = (df2[(df2['homeTeam']  == team) | (df2['awayTeam']  == team)].index.tolist())
                gamesIndex.append([i[0] for i in indexes])

            # gets label positions as int so .iloc and .iat can be used
            iHTGS = df.columns.get_loc('HTGS')
            iATGS = df.columns.get_loc('ATGS')
            iHTGC = df.columns.get_loc('HTGC')
            iATGC = df.columns.get_loc('ATGC')
            iHT = df.columns.get_loc('homeTeam')
            iAT = df.columns.get_loc('awayTeam')

            goalsScored = get_goalsScored(df.loc[(slice(None),seasonNames[season]),:], nMatchWeeks)
            goalsConceded = get_goalsConceded(df.loc[(slice(None),seasonNames[season]),:], nMatchWeeks)

            for i in range(nTeams):
                matchWeek = 0
                for j in range(nMatchWeeks):
                    ht = df.iloc[int(gamesIndex[i][j])].homeTeam
                    at = df.iloc[int(gamesIndex[i][j])].awayTeam
                    df.iat[gamesIndex[i][j],iHTGS] = goalsScored.loc[ht][matchWeek]
                    df.iat[gamesIndex[i][j],iATGS] = goalsScored.loc[at][matchWeek]
                    df.iat[gamesIndex[i][j],iHTGC] = goalsConceded.loc[ht][matchWeek]
                    df.iat[gamesIndex[i][j],iATGC] = goalsConceded.loc[ht][matchWeek]    

                    matchWeek += 1
        except:
            skippedSeasons.append(season)
            pass 
    return (df, skippedSeasons)

### Call

In [21]:
if 'HTGS' not in df:
    df = df.reindex(columns = df.columns.tolist() + ['HTGS','ATGS', 'HTGC', 'ATGC'])

df, skippedSeasons1 = get_aggGoals(df)

### a few (11) seasons don't work and should therefore be droped, they are in list 'skippedSeasons1'

**Cup or playoff-series**
'A1 2009/2010',
'Trophee des Champions 2014',
'Trophee des Champions 2015',
'Trophee des Champions 2016',
'Trophee des Champions 2017',
'Trophee des Champions 2018'

**Unknow, buggcheck remains**
'1. Division Women 2010/2011',
 '1. Division Women 2011/2012',
 '1. Division Women 2012/2013',
 '1. Division Women 2013/2014',
 '2. Bundesliga 2015/2016',

In [22]:
df1 = df.copy()

# Get points

In [23]:
#RELOAD dataFrame
df = df1.copy()

### Helper functions

In [24]:
def get_points(result):
    if result == 'W':
        return 2
    elif result == 'D':
        return 1
    else:
        return 0

def get_cuml_points(matchres, ROUNDS, TEAMS):
    matchres_points = matchres.applymap(get_points)
    for i in range(2, ROUNDS + 1):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]

    matchres_points.insert(column=0, loc=0, value=[0*i for i in range(TEAMS)])
    return matchres_points
    
def get_matchres(frame, ROUNDS):
    # Create a dictionary with team names as keys
    teams = {}
    for i in frame.groupby('homeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(frame)):
        if frame.iloc[i].ftr == 'Home':
            teams[frame.iloc[i].homeTeam].append('W')
            teams[frame.iloc[i].awayTeam].append('L')
        elif frame.iloc[i].ftr == 'Away':
            teams[frame.iloc[i].awayTeam].append('W')
            teams[frame.iloc[i].homeTeam].append('L')
        else:
            teams[frame.iloc[i].awayTeam].append('D')
            teams[frame.iloc[i].homeTeam].append('D')

    return pd.DataFrame(data=teams, index=[i for i in range(1, ROUNDS + 1)]).T

### Main function get_aggPoints

In [25]:
def get_aggPoints(df): 
    # gets label positions as int so .iloc and .iat can be used MOVE TO BERFOR FORLOOP
    iHTP = df.columns.get_loc('HTP')
    iATP = df.columns.get_loc('ATP')
    iHT = df.columns.get_loc('homeTeam')
    iAT = df.columns.get_loc('awayTeam')
    
    skippedSeasons = []
    for season in range(len(seasonNames)):
        try:
            nMatchWeeks = get_rounds(df.loc[(slice(None),seasonNames[season]),:])
            nTeams = get_teams(df.loc[(slice(None),seasonNames[season]),:])

            teams = df.loc[(slice(None),seasonNames[season]),:].homeTeam.unique().tolist()
            gamesIndex = []
            for team in teams:
                df2 = df.loc[(slice(None),seasonNames[season]),:]
                indexes = (df2[(df2['homeTeam']  == team) | (df2['awayTeam']  == team)].index.tolist())
                gamesIndex.append([i[0] for i in indexes])

            matchres = get_matchres(df.loc[(slice(None), seasonNames[season]),:], nMatchWeeks)
            cum_pts = get_cuml_points(matchres, nMatchWeeks, nTeams)
            
            
            for i in range(nTeams):
                matchWeek = 0
                for j in range(nMatchWeeks):
                    ht = df.iloc[int(gamesIndex[i][j])].homeTeam
                    at = df.iloc[int(gamesIndex[i][j])].awayTeam
                    df.iat[gamesIndex[i][j],iHTP] = cum_pts.loc[ht][matchWeek]
                    df.iat[gamesIndex[i][j],iATP] = cum_pts.loc[at][matchWeek]    

                    matchWeek += 1
                    
        except:
            skippedSeasons.append(season)
            pass 
        
    return (df, skippedSeasons)

### Call get_aggPoints

In [26]:
if 'HTP' not in df:
    df = df.reindex(columns = df.columns.tolist() + ['HTP','ATP'])
df, iteration1 = get_aggPoints(df)

### SIDE DEV TO FIND HIGHEST ODDS

In [ ]:
nGames = len(df.loc[df['country'] == 'Sweden'])
gamesIndex = []
for i in range(nGames):
    games = df.loc[df['country'] == 'Sweden'].copy()
    indexes = games.index.tolist()
    gamesIndex.append([i[0] for i in indexes])

## Get XX 

In [269]:
# DEV to REALOAD
df = df1.copy()

### Helpers

## Main function

In [498]:
def get_mw(df):
    if 'HMW' not in df:
        df['HMW'] = ''
    if 'AMW' not in df:
        df['AMW'] = ''    
    iHMW = df.columns.get_loc('HMW')
    iAMW = df.columns.get_loc('AMW')
    iHT = df.columns.get_loc('homeTeam')
    iAT = df.columns.get_loc('awayTeam')

    skippedSeasons = []
    for season in range(len(seasonNames)):
        try:       
            nMatchWeeks = get_rounds(df.loc[(slice(None),seasonNames[season]),:])
            nTeams = get_teams(df.loc[(slice(None),seasonNames[season]),:])
            teams = df.loc[(slice(None),seasonNames[season]),:].homeTeam.unique().tolist()
            gamesIndex = []
            for team in teams:
                df2 = df.loc[(slice(None),seasonNames[season]),:]
                indexes = (df2[(df2['homeTeam']  == team) | (df2['awayTeam']  == team)].index.tolist())
                gamesIndex.append([i[0] for i in indexes])
            i = 0
            for team in teams:
                for j in range(nMatchWeeks):
                    if team == df.iat[gamesIndex[i][j],iHT]:
                        df.iat[gamesIndex[i][j],iHMW] = j+1
                    if team == df.iat[gamesIndex[i][j],iAT]:
                        df.iat[gamesIndex[i][j],iAMW] = j+1
                i += 1
        except:
            skippedSeasons.append(season)
            pass
    
    df.AMW = pd.to_numeric(df.AMW)
    df.HMW = pd.to_numeric(df.HMW)
    df['MW'] = df[['HMW', 'AMW']].mean(axis=1)
        
    return (df, skippedSeasons)

## Call main

In [272]:
df, skippedSeasons3 = get_mw(df)

## Get form points

## Get Form

### Helper Functions add_form

In [442]:
def get_form(df, ROUNDS, num):
    form = get_matchres(df, ROUNDS)
    form_final = form.copy()
    for i in range(num, ROUNDS+1):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

def cleanUp_gameForm(df):
    mask = df.HM2.apply(len) < 2
    df.loc[mask, ['HM2','HM3','HM4','HM5']] = df['HM1']

    mask = df.HM3.apply(len) < 3
    df.loc[mask, ['HM3','HM4','HM5']] = df['HM2']

    mask = df.HM4.apply(len) < 4
    df.loc[mask, ['HM4','HM5']] = df['HM3']

    mask = df.HM5.apply(len) < 5
    df.loc[mask, ['HM5']] = df['HM4']

    mask = df.AM2.apply(len) < 2
    df.loc[mask, ['AM2','AM3','AM4','AM5']] = df['AM1']

    mask = df.AM3.apply(len) < 3
    df.loc[mask, ['AM3','AM4','AM5']] = df['AM2']

    mask = df.AM4.apply(len) < 4
    df.loc[mask, ['AM4','AM5']] = df['AM3']

    mask = df.AM5.apply(len) < 5
    df.loc[mask, ['AM5']] = df['AM4']
    
    return df

def cut_string(df):
    df['HM2'] = df1['HM2'].str[-1:]
    df['HM3'] = df1['HM3'].str[-1:]
    df['HM4'] = df1['HM4'].str[-1:]
    df['HM5'] = df1['HM5'].str[-1:]
    df['AM2'] = df1['AM2'].str[-1:]
    df['AM3'] = df1['AM3'].str[-1:]
    df['AM4'] = df1['AM4'].str[-1:]
    df['AM5'] = df1['AM5'].str[-1:]
    return df

### Main function

In [434]:
def add_form(df, num):
    if ('HM' + str(num)) not in df:
        df['HM' + str(num)] = ''

    if ('AM' + str(num)) not in df:
        df['AM' + str(num)] = ''
        
    iHMnum = df.columns.get_loc('HM' + str(num))
    iAMnum = df.columns.get_loc('AM' + str(num))

    skippedSeasons = []
    for season in range(len(seasonNames)):
    #for season in range(2):
        try:
            nMatchWeeks = get_rounds(df.loc[(slice(None),seasonNames[season]),:])
            nTeams = get_teams(df.loc[(slice(None),seasonNames[season]),:])

            teams = df.loc[(slice(None),seasonNames[season]),:].homeTeam.unique().tolist()
            gamesIndex = []
            for team in teams:
                df2 = df.loc[(slice(None),seasonNames[season]),:]
                indexes = (df2[(df2['homeTeam']  == team) | (df2['awayTeam']  == team)].index.tolist())
                gamesIndex.append([i[0] for i in indexes])

            form = get_form(df.loc[(slice(None),seasonNames[season]),:], nMatchWeeks, num)

            for i in range(nTeams):
                matchWeek = 1
                for j in range(1,nMatchWeeks):
                    ht = df.iloc[int(gamesIndex[i][j])].homeTeam
                    at = df.iloc[int(gamesIndex[i][j])].awayTeam

                    df.iat[gamesIndex[i][j],int(iHMnum)] = form.loc[ht][matchWeek]
                    df.iat[gamesIndex[i][j],int(iAMnum)] = form.loc[at][matchWeek]

                    matchWeek += 1
        except:
            skippedSeasons.append(season)
            pass
        
    return (df, skippedSeasons)

### Call main add_form

In [435]:
nGamesToGetFormFrome = 6
for num in range(1,nGamesToGetFormFrome):
    df, skippedSeasons2 = add_form(df, num)
df = cleanUp_gameForm(df)
df = cut_string(df)

,,country,date,homeTeam,awayTeam,score,homeScore,awayScore,odds1,oddsX,odds2,...,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
,season,,,,,,,,,,,,,,,,,,,,,
0,1. Bundesliga Women 2008/2009,Germany,2008-09-05,Goppingen W,Nurnberg W,29:25,29,25,NaN,NaN,NaN,...,,,,,,,,,,
1,1. Bundesliga Women 2008/2009,Germany,2008-09-06,BVB Dortmund W,Leverkusen W,19:28,19,28,NaN,NaN,NaN,...,,,,,,,,,,
2,1. Bundesliga Women 2008/2009,Germany,2008-09-06,DJK Trier W,Rhein-Main Bienen W,34:23,34,23,NaN,NaN,NaN,...,,,,,,,,,,
3,1. Bundesliga Women 2008/2009,Germany,2008-09-06,Frankfurt W,Blomberg-Lippe W,30:24,30,24,NaN,NaN,NaN,...,,,,,,,,,,
4,1. Bundesliga Women 2008/2009,Germany,2008-09-07,Leipzig W,Thuringer W,39:24,39,24,NaN,NaN,NaN,...,,,,,,,,,,


In [499]:
df1 = df.copy()

In [507]:
df = df1.copy()

In [508]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

df['HTFormPtsStr'] = df['HM1'] + df['HM2'] + df['HM3'] + df['HM4'] + df['HM5']
df['ATFormPtsStr'] = df['AM1'] + df['AM2'] + df['AM3'] + df['AM4'] + df['AM5']

df['HTFormPts'] = df['HTFormPtsStr'].apply(get_form_points)
df['ATFormPts'] = df['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
df['HTWinStreak3'] = df['HTFormPtsStr'].apply(get_3game_ws)
df['HTWinStreak5'] = df['HTFormPtsStr'].apply(get_5game_ws)
df['HTLossStreak3'] = df['HTFormPtsStr'].apply(get_3game_ls)
df['HTLossStreak5'] = df['HTFormPtsStr'].apply(get_5game_ls)

df['ATWinStreak3'] = df['ATFormPtsStr'].apply(get_3game_ws)
df['ATWinStreak5'] = df['ATFormPtsStr'].apply(get_5game_ws)
df['ATLossStreak3'] = df['ATFormPtsStr'].apply(get_3game_ls)
df['ATLossStreak5'] = df['ATFormPtsStr'].apply(get_5game_ls)

df['DiffFormPts'] = df['HTFormPts'] - df['ATFormPts']

In [492]:
'HTWinStreak3','HTWinStreak5','HTLossStreak3','HTLossStreak5','ATWinStreak3','ATWinStreak5','ATLossStreak3','ATLossStreak5'

,,country,date,homeTeam,awayTeam,score,homeScore,awayScore,odds1,oddsX,odds2,...,ATFormPts,HTWinStreak3,HTWinStreak5,HTLossStreak3,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,DiffFormPts
,season,,,,,,,,,,,,,,,,,,,,,
0,1. Bundesliga Women 2008/2009,Germany,2008-09-05,Goppingen W,Nurnberg W,29:25,29,25,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1. Bundesliga Women 2008/2009,Germany,2008-09-06,BVB Dortmund W,Leverkusen W,19:28,19,28,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1. Bundesliga Women 2008/2009,Germany,2008-09-06,DJK Trier W,Rhein-Main Bienen W,34:23,34,23,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1. Bundesliga Women 2008/2009,Germany,2008-09-06,Frankfurt W,Blomberg-Lippe W,30:24,30,24,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1. Bundesliga Women 2008/2009,Germany,2008-09-07,Leipzig W,Thuringer W,39:24,39,24,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,1. Bundesliga Women 2008/2009,Germany,2008-09-07,Oldenburg W,Buxtehuder SV W,22:17,22,17,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6,1. Bundesliga Women 2008/2009,Germany,2008-09-13,DJK Trier W,Buxtehuder SV W,21:26,21,26,NaN,NaN,NaN,...,0,1,1,0,0,0,0,1,1,10
7,1. Bundesliga Women 2008/2009,Germany,2008-09-13,Oldenburg W,Nurnberg W,24:27,24,27,NaN,NaN,NaN,...,0,1,1,0,0,0,0,1,1,10
8,1. Bundesliga Women 2008/2009,Germany,2008-09-13,Rhein-Main Bienen W,BVB Dortmund W,31:21,31,21,NaN,NaN,NaN,...,0,0,0,1,1,0,0,1,1,0


In [509]:
# Get Goal Difference
df['HTGD'] = df['HTGS'] - df['HTGC']
df['ATGD'] = df['ATGS'] - df['ATGC']

# Diff in points
df['DiffPts'] = df['HTP'] - df['ATP']

# Diff in goal difference
df['DiffGD'] = df['HTGD'] - df['ATGD']


In [510]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','DiffGD','HTP','ATP']


for col in cols:
    df[col] = df[col] / df.MW

In [513]:
cols = ['homeTeam','awayTeam','score','ftr','DiffPts','DiffFormPts','DiffGD']
df[cols]

,,homeTeam,awayTeam,score,ftr,DiffPts,DiffFormPts,DiffGD
,season,,,,,,,
0,1. Bundesliga Women 2008/2009,Goppingen W,Nurnberg W,29:25,Home,0.000000,0.000000,0.000000
1,1. Bundesliga Women 2008/2009,BVB Dortmund W,Leverkusen W,19:28,Away,0.000000,0.000000,0.000000
2,1. Bundesliga Women 2008/2009,DJK Trier W,Rhein-Main Bienen W,34:23,Home,0.000000,0.000000,0.000000
3,1. Bundesliga Women 2008/2009,Frankfurt W,Blomberg-Lippe W,30:24,Home,0.000000,0.000000,0.000000
4,1. Bundesliga Women 2008/2009,Leipzig W,Thuringer W,39:24,Home,0.000000,0.000000,0.000000
5,1. Bundesliga Women 2008/2009,Oldenburg W,Buxtehuder SV W,22:17,Home,0.000000,0.000000,0.000000
6,1. Bundesliga Women 2008/2009,DJK Trier W,Buxtehuder SV W,21:26,Away,1.000000,5.000000,8.500000
7,1. Bundesliga Women 2008/2009,Oldenburg W,Nurnberg W,24:27,Away,1.000000,5.000000,-1.500000
8,1. Bundesliga Women 2008/2009,Rhein-Main Bienen W,BVB Dortmund W,31:21,Home,0.000000,0.000000,2.000000


## Features selection
Alt a
1. Diff Points
2. Diff Form Points
3. Diff Goal Difference

Alt b
1. Odds 1
2. Odds x
3. Odds 2
4. HM 1 - 5
   AM 1 - 5
6. Diff Points
7. Diff Form Points
--




In [515]:
df.to_pickle('stats.pkl') 

In [ ]:
df = pd.read_pickle('stats.pkl')